# Predicting if Someone has Tried Cocaine
## Model Building

You can also check out my Crack/Cocaine Usage Prediction repository on my Github!

https://github.com/bgallamoza/Cocaine_Usage_Classification

In [ ]:
import pandas as pd
import numpy as np
import pickle

# Data Preprocessing

Before we can use our data to train models, we need to do a few things:

1. Select our desired features

2. Apply standard scaling to our numerical variables

3. Dummify/One Hot Encode our categorical variables

We begin with removing coutyp4, year, irwrkstat, and mjever columns. These columns were only used for EDA or do not seem to provide insight in target prediction.

In [ ]:
# Read in pickled data, and drop unneeded columns
df = pd.read_pickle("../input/crack-cocaine-usage-prediction-eda/NSDUH_cleaned_dropna_2016-2019.pkl")
df = df.drop(['cocever', 'crkever', 'year'], axis=1)
df

Now, we must separate our numerical and categorical variables. Numerical variables will be adjusted per column by StandardScaler(), which converts the data such that the mean and standard deviation is 0 and 1 for that column, respectively. This standardization across numerical variables increases our model's accuracy.

As for categorical variables, each unique value in a categorical column must be given its own separate, binary column indicating if that observation fits that unique value or not. We do this because keeping them in one column implies some kind of order. Something like race (newrace2 column) makes no sense to order, and is therefore a candidate to be separated into different columns (dummified). 

In [ ]:
# Continuous variables
num_cols = [
    "iralcfy",
    "catag3",
    "health",
    "ireduhighst2",
    "irpinc3",
    "irki17_2",
    "irmjfy",
    "wrkdhrswk2",
    'irhhsiz2',
    'cig30use',
    'irherfy',
    'irmethamyfq'
]

# Categorical variables
cat_cols = [
    "newrace2",
    "irsex"
]

# Data Preprocessing with Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
# Create a preprocessor from ColumnTransformer
# StandardScaler() applied to num_cols, and OneHotEncoder() applied to cat_cols
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first'), cat_cols)
])

# Model Building

Now that our data is properly processed, we can test several models across different hyperparameters using GridSearchCV. We will be testing the following models:

1. RandomForestClassifier()

2. LogisticRegression()

3. svm.LinearSVC()

*We will only test the default hyperparamters for our LinearSVC due to the length of time required to train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split

In [ ]:
# Define feature and target columns
features = num_cols+cat_cols
target = "coccrkever"

# Standard naming conventions for feature/test datasets
X = df[features]
y = df[target]

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first'), cat_cols)
])

# Parameter grid for GridSearchCV
model_grid = {
    'random_forest': {
        'model':RandomForestClassifier(random_state=15, n_jobs=3, n_estimators=500),
        'params': {
            'estimator__max_depth': [11, 12, 13, 14],
            'estimator__criterion':['gini', 'entropy']
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(random_state=15, n_jobs=3),
        'params': {
            'estimator__C': [0.085, 0.09, 0.092],
            'estimator__solver':['lbfgs', 'liblinear'],
        }
    },
    'svm': {
        'model': svm.LinearSVC(random_state=15, max_iter=100000),
        'params': {
            'estimator__C':[0.52, 0.55, 0.6, 0.65]
        }
    }
}

In [ ]:
# List to hold model scores
scores = []

for model_name, model_params in model_grid.items():
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('estimator', model_params['model'])
    ])

    model = GridSearchCV(estimator=pipe, param_grid=model_params['params'], cv=4, return_train_score=False, refit=True)
    model.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score:': model.best_score_,
        'best_params': model.best_params_
    })

In [ ]:
# Show scores as df
scores

# Choosing a Model

Although our random forest model has the highest accuracy, the accuracies are very similar. Let's train a model using the best hyperparameters of each, then look at the classification report of each model to gain better insight into the performance of each model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.25, random_state=12)

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first'), cat_cols)
])

pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
    ])

X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)

In [ ]:
# Train a model for each model type using our best hyperparameters. This is so we can analyze each
# in a classification report

rf = RandomForestClassifier(random_state=15, n_jobs=3, n_estimators=500, max_depth=14, criterion='entropy')
rf.fit(X_train, y_train)

lg = LogisticRegression(random_state=15, solver='liblinear',C=0.085)
lg.fit(X_train, y_train)

lsvc = svm.LinearSVC(random_state=15, max_iter=100000, C=0.52)
lsvc.fit(X_train, y_train)

In [ ]:
rf_predict = rf.predict(X_test)
lg_predict = lg.predict(X_test)
lsvc_predict = lsvc.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
print("Random Forest Score: %f\nLogistic Regression Score: %f\nLinear SVC Score: %f\n" %(accuracy_score(y_test, rf_predict), accuracy_score(y_test, lg_predict), accuracy_score(y_test, lsvc_predict)))
print("Random Forest:\n", classification_report(y_test, rf_predict))
print("Logistic Regression:\n", classification_report(y_test, lg_predict))
print("Linear SVC:\n", classification_report(y_test, lsvc_predict))

## Accuracy vs Precision vs Recall

Although the random forest performed the best in terms of total accuracy, our linear SVC model has the highest precision of each model. Recall teh differences between accuracy, precision, and recall:

1. **Accuracy**: Proportion of correct predictions from total observations

2. **Precision**: For a given class, the proportion of correct predictions from total predictions

3. **Recall**: For a given class, proportion of correct predictions from the total number of true observations for that class

Our models have low recall. That means we miss a large number of people who have actually used cocaine. However, we also have extremely high precision. This means that for the people we *do* predict have used cocaine, we are actually correct! This is important to consider. If your goal is to either help people using cocaine or prevent people from becoming addicted cocaine, it would be very bad to wrongly approach someone believing they've tried cocaine when they actually have not. **To prevent false positives, we will choose our Random Forest Classifier because it has the highest precision.**

# Saving our Data
Although we've decided on using the linear SVC model, we will save all the models regardless, just in case we want them in the future.

In [ ]:
# Pickle models for later
for model, name in zip([lg, lsvc], ["logreg_model", "lsvc_model"]):
    with open("./" + name + ".pickle", 'wb') as f:
        pickle.dump(model, f)

In [ ]:
import gzip, pickletools

# The output of a regular pickle.dump for our random forest is quite large,
# we can compress it using gzip
with gzip.open("./randforest_model.pickle", "wb") as f:
    pickled = pickle.dumps(rf)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)

"""Code for loading from a gzipped pickle file"""
# with gzip.open("./randforest_model_optimized.pickle", 'rb') as f:
#     p = pickle.Unpickler(f)
#     rf = p.load()

Next, we need to save our fitted pipeline to transform future data.

In [ ]:
# Pickle our Random Forest Classifier
with open("./pipeline.pickle", 'wb') as f:
        pickle.dump(pipe, f)

Finally, let's save our columns as a JSON file for future reference.

In [ ]:
import json

# Firstly we append our data columns to the json
column_info = {
    'data_columns' : [col for col in num_cols+cat_cols]
}

# Write dict to a json file
with open("./data_columns.json", "w") as f:
    f.write(json.dumps(column_info))

Now, we can move on to creating a server where we can make our model easily interactable.